<a href="https://colab.research.google.com/github/2598Nitz/MicrosoftAI/blob/master/Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1FbE-bTxaeIyhevzXQkxsVLwmIvGvnIu3'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('cleandata1.tsv')


In [0]:
downloaded = drive.CreateFile({'id':'1T50rUZMbFcNjsXG8gpEC-rc5xg6dS1kt'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('cleaneval2.tsv')

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('cleandata1.tsv',sep = "\t", names = ['query_id','query','passage','label','passage_id'])

In [0]:
testdf = pd.read_csv('cleaneval2.tsv',sep = "\t", names = ['query_id','query','passage','passage_id'])

In [0]:
df.shape

(5241880, 5)

In [0]:
downloaded = drive.CreateFile({'id':'1Ed3HMlRJD_54CebqKiMHydjgAYTl3Ndh'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('train.tsv')
downloaded = drive.CreateFile({'id':'11PUImJZ6IvGNmKS_EV6jgQYeHr4GhLGs'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('test.tsv')

In [0]:
dfr = pd.read_csv('train.tsv',sep = "\t", names = ['bm25'])
dfe = pd.read_csv('test.tsv',sep = "\t", names = ['bm25'])

In [0]:
dfr.shape

(5241880, 1)

In [0]:
dfe.shape

(415380, 1)

In [0]:
df['bm25'] = dfr['bm25'].copy()
testdf['bm25'] = dfe['bm25'].copy()

In [0]:
testdf.shape

(415380, 5)

In [0]:
pos_df = df[df.label == 1]
neg_df = df[df.label == 0].sample(frac = 0.3)

In [0]:
new_df = pd.concat([pos_df,neg_df],ignore_index = True)

In [0]:
shuffle_df = new_df.sample(frac=1).reset_index(drop=True)

Feature Extraction:
TFIDF matrix,
BM25 score,
No. of words,
No. of characters,
NER ? 
Embedding Matrix?,
bigram matrix?,
Distances from Embeddings

In [0]:
train_valid = shuffle_df.iloc[:450000,:].copy()

In [0]:
train_valid['query_char_len'] = train_valid['query'].str.len()
train_valid['passage_char_len'] = train_valid['passage'].str.len()

In [0]:
train_valid['query_word_len'] = train_valid['query'].str.split().str.len()
train_valid['passage_word_len'] = train_valid['passage'].str.split().str.len()

In [0]:
train_valid['char_ratio'] = train_valid['passage_char_len']/train_valid['query_char_len']
train_valid['word_ratio'] = train_valid['passage_word_len']/train_valid['query_word_len']

In [0]:
def f(columns):
  f_desc, f_def = columns[1], columns[2]
  common = set(f_desc.split()).intersection(set(f_def.split()))
  return len(common)

In [0]:
train_valid['overlap_count'] = train_valid.apply(f, axis=1, raw=True).apply(pd.Series)

In [0]:
downloaded = drive.CreateFile({'id':'1tTmmlA7XgKNS3CFrpnVmMGLcq_kKLsGD'}) # replace fileid with Id of file you want to access
downloaded.GetContentFile('GoogleNews-vectors-negative300.bin')

In [0]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
model.init_sims(replace=True)

In [0]:
train_valid['wmd'] = train_valid.apply(lambda row: model.wmdistance(row.query,row.passage), axis = 1)

In [0]:
train_valid.drop(['query_id','passage_id'],axis = 1,inplace = True)

In [0]:
print("\nDtypes of DF features:\n",train_valid.dtypes.value_counts())


Dtypes of DF features:
 int64      6
float64    3
object     2
dtype: int64


In [0]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    if(len(a) == 0 or len(b) == 0):
      return 0.0
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [0]:
train_valid['jaccard_sim'] = train_valid.apply(lambda row: get_jaccard_sim(row.query,row.passage), axis = 1)

In [0]:
train_valid.to_csv('trainxgb.tsv', sep = '\t', index = False)

In [0]:
#!wget https://github.com/nlptown/nlp-notebooks/blob/master/data/sentence_similarity/doc_frequencies.tsv

In [0]:
#!wget https://github.com/nlptown/nlp-notebooks/blob/master/data/sentence_similarity/frequencies.tsv

In [0]:
# downloaded = drive.CreateFile({'id':'10Rtte1j-FA9kpIT1pAg67TwxtW9pD4hW'}) # replace fileid with Id of file you want to access
# downloaded.GetContentFile('glove.6B.100d.txt')


Rest distances will be obtained later. Continue with XGboost pipeline

In [0]:
train_valid.head(10)

In [0]:
testdf['query'] = testdf['query'].str.replace('[\[,\]]|\'', '')
testdf['passage'] = testdf['passage'].str.replace('[\[,\]]|\'', '')

In [0]:
testdf['query_char_len'] = testdf['query'].str.len()
testdf['passage_char_len'] = testdf['passage'].str.len()
testdf['query_word_len'] = testdf['query'].str.split().str.len()
testdf['passage_word_len'] = testdf['passage'].str.split().str.len()
testdf['char_ratio'] = testdf['passage_char_len']/testdf['query_char_len']
testdf['word_ratio'] = testdf['passage_word_len']/testdf['query_word_len']
testdf['overlap_count'] = testdf.apply(f, axis=1, raw=True).apply(pd.Series)
testdf.drop(['query_id','passage_id'],axis = 1,inplace = True)
testdf['jaccard_sim'] = testdf.apply(lambda row: get_jaccard_sim(row.query,row.passage), axis = 1)


In [0]:
testdf['wmd'] = testdf.apply(lambda row: model.wmdistance(row.query,row.passage), axis = 1)

In [0]:
testdf.to_csv('testxgb.tsv', sep = '\t', index = False)

In [0]:
import pandas as pd

In [0]:
train_valid = pd.read_csv('trainxgb.tsv', sep = '\t')

In [0]:
train_valid.head()

In [0]:
test_df = pd.read_csv('testxgb.tsv', sep = '\t')

In [0]:
testdf.head()

Try changing features

In [0]:
train_valid = train_valid.drop(['wmd'],axis = 1)
test_df = test_df.drop(['wmd'],axis = 1)

Sentence encoding

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [0]:
def run_gse_benchmark(sentences1, sentences2):
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    sts_encode1 = tf.nn.l2_normalize(embed(sts_input1))
    sts_encode2 = tf.nn.l2_normalize(embed(sts_input2))
        
    sim_scores = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
      
        [gse_sims] = session.run(
            [sim_scores],
            feed_dict={
                sts_input1: [sentences1],
                sts_input2: [sentences2]
            })
    return gse_sims

In [0]:
test_df['gse'] = test_df.apply(lambda row: run_gse_benchmark(row.query,row.passage), axis = 1)

In [0]:
dummy = test_df.iloc[:10,:].copy()

In [0]:
dummy['gse'] = dummy.apply(lambda row: run_gse_benchmark(row.query,row.passage), axis = 1)

In [0]:
train_valid['text'] = train_valid.apply(lambda row: ' '.join([str(row['query']), str(row['passage'])]), axis=1)
testdf['text'] = testdf.apply(lambda row: ' '.join([str(row['query']), str(row['passage'])]), axis=1)

In [0]:
train_text = train_valid['text'].apply(lambda x: ''.join(x))
test_text = testdf['text'].apply(lambda x: ''.join(x))

In [0]:
train_text[:10]

In [0]:
test_text[:10]

In [0]:
import random
from sklearn import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from scipy.sparse import hstack, vstack, csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb

In [0]:
tfv = TfidfVectorizer(norm='l2', min_df=0,  max_features=8000, 
            strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
            ngram_range=(1,2), use_idf=True, smooth_idf=False, sublinear_tf=True,
            stop_words = 'english')

In [0]:
tfv.fit(list(train_text.values) + list(test_text.values))
train_tfv = tfv.transform(train_text)
test_tfv = tfv.transform(test_text)

In [0]:
train_valid.dtypes.value_counts()

int64      6
float64    4
object     3
dtype: int64

In [0]:
feat_train = train_valid.drop(['query','passage','text','label'], axis = 1).copy()
feat_test = testdf.drop(['query','passage','text'], axis = 1).copy()
target = train_valid['label'].copy()

In [0]:
print(feat_train.dtypes.value_counts())
print(feat_test.dtypes.value_counts())

int64      5
float64    4
dtype: int64
int64      5
float64    4
dtype: int64


In [0]:
feat_train = csr_matrix(feat_train.values)
feat_test = csr_matrix(feat_test.values)
X_train_stack = hstack([feat_train, train_tfv[0:feat_train.shape[0]]])
X_test_stack = hstack([feat_test, test_tfv[0:feat_test.shape[0]]])

print('Train shape: ', X_train_stack.shape, '\n\nTest Shape: ', X_test_stack.shape)

Train shape:  (450000, 8009) 

Test Shape:  (415380, 8009)


In [0]:
seed = 28

In [0]:
K = 5 # How many folds do you want? 
kf = KFold(n_splits = K, random_state = seed, shuffle = True)

In [0]:
cv_scores = []
xgb_preds = []

for train_index, test_index in kf.split(X_train_stack):
    
    # Split out a validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_stack, target, test_size=0.20, random_state=random.seed(seed))
    
    # params are tuned with kaggle kernels in mind
    xgb_params = {'eta': 0.15, 
                  'max_depth': 6, 
                  'subsample': 0.80, 
                  'colsample_bytree': 0.80, 
                  'objective': 'binary:logistic', 
                  'eval_metric': 'auc', 
                  'seed': seed
                 }
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    d_test = xgb.DMatrix(X_test_stack)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 2000, watchlist, verbose_eval=50, early_stopping_rounds=30)
    cv_scores.append(float(model.attributes()['best_score']))
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

In [0]:
import numpy as np
print(cv_scores)
print(np.mean(cv_scores))

[0.710728, 0.709724, 0.711833, 0.711991, 0.71191]
0.7112372


In [0]:
x_preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    x_preds.append(sum / K)

In [0]:
testdf = pd.read_csv('cleaneval2.tsv',sep = "\t", names = ['query_id','query','passage','passage_id'])

In [0]:
k = 0
f = open('answer_xgb.tsv','w')
for index,row in testdf.iterrows():
  if index%10 == 0:
    newlist = []
    for val in x_preds[k:k+10]:
      newlist.append(val)
    k = k+10
    scores_str = [str(v) for v in newlist]
    scores_str = "\t".join(scores_str)
    f.write(str(row['query_id'])+"\t"+scores_str+"\n")